### Restart and run all cells

### stock: dividend, price

In [1]:
import calendar
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
today

datetime.date(2022, 3, 24)

In [2]:
format_dict = {
               'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
               'price_x':'{:,.2f}','to_price':'{:,.2f}',
               'current':'{:,.2f}','div_amt':'{:,.2f}','amount':'{:,.2f}','yield':'{:,.2f}',
               'Q4':'{:,.2f}','Q3':'{:,.2f}','Q2':'{:,.2f}','Q1':'{:,.2f}','DIVIDEND':'{:,.2f}',
               'qty':'{:,}','shares':'{:,}','qty_z':'{:,}'
              }
pd.set_option("display.max_rows", None)

In [3]:
sql = '''
SELECT *
FROM dividend
ORDER BY name'''
df_dividend = pd.read_sql(sql, const)
df_dividend.shape

(81, 13)

In [4]:
### To find dividend without price record
sql = '''
SELECT D.name, P.name
FROM dividend D
LEFT JOIN price P
ON D.name = P.name
AND P.date = "%s"
WHERE P.name IS NULL
'''
sql = sql % today
print(sql)

df_left = pd.read_sql(sql, const)
df_left


SELECT D.name, P.name
FROM dividend D
LEFT JOIN price P
ON D.name = P.name
AND P.date = "2022-03-24"
WHERE P.name IS NULL



,name,name


### If above statement consists stock names, must delete from dividend as there are no price records

In [5]:
sql = '''
SELECT D.name, FORMAT(DIVIDEND/P.price*100,2) AS yield,
Q4, Q3, Q2, Q1, DIVIDEND, shares, DIVIDEND * shares AS div_amt,
P.price AS current, shares*P.price AS amount, XDATE, PAIDDATE, ACTUAL
FROM dividend D
JOIN price P
WHERE D.name = P.name
AND P.date = "%s"'''
sql = sql % today
print(sql)

div_price = pd.read_sql(sql, const)
div_price['yield'] = div_price['yield'].astype(float)
div_price['shares'] = div_price['shares'].astype(int)
div_price.shape


SELECT D.name, FORMAT(DIVIDEND/P.price*100,2) AS yield,
Q4, Q3, Q2, Q1, DIVIDEND, shares, DIVIDEND * shares AS div_amt,
P.price AS current, shares*P.price AS amount, XDATE, PAIDDATE, ACTUAL
FROM dividend D
JOIN price P
WHERE D.name = P.name
AND P.date = "2022-03-24"


(81, 14)

In [6]:
div_price.nlargest(3, 'div_amt')[['name','div_amt']]

,name,div_amt
50,JASIF,104500.0
80,RCL,72000.0
2,MCS,63500.0


In [7]:
y_ovr_X = div_price['yield'] >= 7.00
div_price[y_ovr_X].sort_values(['yield'],ascending=[False]).style.format(format_dict)

,name,yield,Q4,Q3,Q2,Q1,DIVIDEND,shares,div_amt,current,amount,XDATE,PAIDDATE,ACTUAL
80,RCL,13.19,3.00,1.50,1.50,0.00,6.00,"12,000","72,000.00",45.50,"546,000.00",2022-03-23,2022-05-20,0
35,STGT,12.23,0.65,1.25,1.25,0.00,3.15,0,0.00,25.75,0.00,2022-04-19,2022-05-05,1
13,SCCC,11.65,9.00,0.00,9.00,0.00,18.00,"1,200","21,600.00",154.50,"185,400.00",2022-02-24,2022-04-08,0
54,ASP,11.56,0.20,0.00,0.20,0.00,0.40,0,0.00,3.46,0.00,2022-03-10,2022-05-17,0
26,TMT,11.11,0.70,0.00,0.60,0.00,1.30,"27,000","35,100.00",11.70,"315,900.00",2022-04-18,2022-05-06,1
2,MCS,9.20,0.50,0.37,0.40,0.00,1.27,"50,000","63,500.00",13.80,"690,000.00",2022-03-15,2022-05-05,0
64,WIIK,8.88,0.09,0.00,0.10,0.00,0.19,0,0.00,2.14,0.00,2022-05-05,2022-05-26,1
50,JASIF,8.64,0.25,0.22,0.24,0.24,0.95,"110,000","104,500.00",11.00,"1,210,000.00",2022-03-03,2022-03-21,0
71,HREIT,8.04,0.17,0.17,0.17,0.17,0.70,"40,100","27,877.52",8.65,"346,865.00",2022-03-04,2022-03-30,0
58,KGI,8.03,0.55,0.00,0.00,0.00,0.55,0,0.00,6.85,0.00,2022-04-19,2022-05-05,1


In [8]:
not_due = div_price['ACTUAL'] == 1
div_price[not_due].sort_values(['yield'],ascending=[False]).style.format(format_dict)

,name,yield,Q4,Q3,Q2,Q1,DIVIDEND,shares,div_amt,current,amount,XDATE,PAIDDATE,ACTUAL
35,STGT,12.23,0.65,1.25,1.25,0.00,3.15,0,0.00,25.75,0.00,2022-04-19,2022-05-05,1
26,TMT,11.11,0.70,0.00,0.60,0.00,1.30,"27,000","35,100.00",11.70,"315,900.00",2022-04-18,2022-05-06,1
64,WIIK,8.88,0.09,0.00,0.10,0.00,0.19,0,0.00,2.14,0.00,2022-05-05,2022-05-26,1
58,KGI,8.03,0.55,0.00,0.00,0.00,0.55,0,0.00,6.85,0.00,2022-04-19,2022-05-05,1
79,RJH,7.81,1.00,1.10,0.40,0.00,2.50,"4,500","11,250.00",32.00,"144,000.00",2022-05-09,2022-05-24,1
72,TISCO,7.19,7.15,0.00,0.00,0.00,7.15,"4,000","28,600.00",99.50,"398,000.00",2022-04-29,2022-05-19,1
63,TCAP,7.10,1.80,0.00,1.20,0.00,3.00,0,0.00,42.25,0.00,2022-04-19,2022-05-06,1
37,STA,7.10,0.65,0.00,1.25,0.00,1.90,"22,500","42,750.00",26.75,"601,875.00",2022-04-19,2022-05-06,1
14,DCC,6.21,0.04,0.04,0.05,0.05,0.18,0,0.00,2.98,0.00,2022-04-05,2022-04-28,1
15,NER,6.10,0.36,0.00,0.07,0.00,0.43,"27,000","11,610.00",7.05,"190,350.00",2022-04-19,2022-05-06,1


In [9]:
div_price[y_ovr_X & not_due].sort_values(['XDATE','name'],ascending=[True,True]).style.format(format_dict)

,name,yield,Q4,Q3,Q2,Q1,DIVIDEND,shares,div_amt,current,amount,XDATE,PAIDDATE,ACTUAL
26,TMT,11.11,0.70,0.00,0.60,0.00,1.30,"27,000","35,100.00",11.70,"315,900.00",2022-04-18,2022-05-06,1
58,KGI,8.03,0.55,0.00,0.00,0.00,0.55,0,0.00,6.85,0.00,2022-04-19,2022-05-05,1
37,STA,7.10,0.65,0.00,1.25,0.00,1.90,"22,500","42,750.00",26.75,"601,875.00",2022-04-19,2022-05-06,1
35,STGT,12.23,0.65,1.25,1.25,0.00,3.15,0,0.00,25.75,0.00,2022-04-19,2022-05-05,1
63,TCAP,7.10,1.80,0.00,1.20,0.00,3.00,0,0.00,42.25,0.00,2022-04-19,2022-05-06,1
72,TISCO,7.19,7.15,0.00,0.00,0.00,7.15,"4,000","28,600.00",99.50,"398,000.00",2022-04-29,2022-05-19,1
64,WIIK,8.88,0.09,0.00,0.10,0.00,0.19,0,0.00,2.14,0.00,2022-05-05,2022-05-26,1
79,RJH,7.81,1.00,1.10,0.40,0.00,2.50,"4,500","11,250.00",32.00,"144,000.00",2022-05-09,2022-05-24,1


In [10]:
in_port = div_price['shares'] > 0
div_price[in_port].sort_values(['name'],ascending=[True]).style.format(format_dict)

,name,yield,Q4,Q3,Q2,Q1,DIVIDEND,shares,div_amt,current,amount,XDATE,PAIDDATE,ACTUAL
16,AIMIRT,6.89,0.22,0.22,0.22,0.22,0.88,"10,000","8,822.00",12.80,"128,000.00",2022-03-14,2022-03-28,0
57,BCH,5.61,1.00,0.00,0.20,0.00,1.20,"15,000","18,000.00",21.40,"321,000.00",2022-05-10,2022-05-25,1
70,CPNCG,7.86,0.22,0.24,0.24,0.25,0.96,"20,000","19,174.00",12.20,"244,000.00",2022-02-14,2022-03-04,0
32,DIF,7.35,0.26,0.26,0.26,0.26,1.04,"50,000","52,200.00",14.20,"710,000.00",2022-02-10,2022-03-04,0
73,DOHOME,0.00,0.00,0.00,0.00,0.00,0.00,"12,000",0.00,21.30,"255,600.00",2022-02-02,2022-02-02,0
71,HREIT,8.04,0.17,0.17,0.17,0.17,0.70,"40,100","27,877.52",8.65,"346,865.00",2022-03-04,2022-03-30,0
50,JASIF,8.64,0.25,0.22,0.24,0.24,0.95,"110,000","104,500.00",11.00,"1,210,000.00",2022-03-03,2022-03-21,0
46,KCE,2.45,1.00,0.00,0.60,0.00,1.60,"13,000","20,800.00",65.25,"848,250.00",2022-03-21,2022-05-09,0
5,MAKRO,1.82,0.32,0.00,0.40,0.00,0.72,"3,000","2,160.00",39.50,"118,500.00",2022-03-03,2022-05-19,0
2,MCS,9.20,0.50,0.37,0.40,0.00,1.27,"50,000","63,500.00",13.80,"690,000.00",2022-03-15,2022-05-05,0


In [11]:
div_price[y_ovr_X & in_port].sort_values(['yield','name'],ascending=[False,True]).style.format(format_dict)

,name,yield,Q4,Q3,Q2,Q1,DIVIDEND,shares,div_amt,current,amount,XDATE,PAIDDATE,ACTUAL
80,RCL,13.19,3.00,1.50,1.50,0.00,6.00,"12,000","72,000.00",45.50,"546,000.00",2022-03-23,2022-05-20,0
13,SCCC,11.65,9.00,0.00,9.00,0.00,18.00,"1,200","21,600.00",154.50,"185,400.00",2022-02-24,2022-04-08,0
26,TMT,11.11,0.70,0.00,0.60,0.00,1.30,"27,000","35,100.00",11.70,"315,900.00",2022-04-18,2022-05-06,1
2,MCS,9.20,0.50,0.37,0.40,0.00,1.27,"50,000","63,500.00",13.80,"690,000.00",2022-03-15,2022-05-05,0
50,JASIF,8.64,0.25,0.22,0.24,0.24,0.95,"110,000","104,500.00",11.00,"1,210,000.00",2022-03-03,2022-03-21,0
71,HREIT,8.04,0.17,0.17,0.17,0.17,0.70,"40,100","27,877.52",8.65,"346,865.00",2022-03-04,2022-03-30,0
70,CPNCG,7.86,0.22,0.24,0.24,0.25,0.96,"20,000","19,174.00",12.20,"244,000.00",2022-02-14,2022-03-04,0
79,RJH,7.81,1.00,1.10,0.40,0.00,2.50,"4,500","11,250.00",32.00,"144,000.00",2022-05-09,2022-05-24,1
21,RATCH,7.61,1.35,0.00,2.00,0.00,3.35,"6,000","20,100.00",44.00,"264,000.00",2022-03-17,2022-05-20,0
32,DIF,7.35,0.26,0.26,0.26,0.26,1.04,"50,000","52,200.00",14.20,"710,000.00",2022-02-10,2022-03-04,0


In [12]:
div_price[y_ovr_X & in_port].sort_values(['div_amt','name'],ascending=[False,True]).style.format(format_dict)

,name,yield,Q4,Q3,Q2,Q1,DIVIDEND,shares,div_amt,current,amount,XDATE,PAIDDATE,ACTUAL
50,JASIF,8.64,0.25,0.22,0.24,0.24,0.95,"110,000","104,500.00",11.00,"1,210,000.00",2022-03-03,2022-03-21,0
80,RCL,13.19,3.00,1.50,1.50,0.00,6.00,"12,000","72,000.00",45.50,"546,000.00",2022-03-23,2022-05-20,0
2,MCS,9.20,0.50,0.37,0.40,0.00,1.27,"50,000","63,500.00",13.80,"690,000.00",2022-03-15,2022-05-05,0
32,DIF,7.35,0.26,0.26,0.26,0.26,1.04,"50,000","52,200.00",14.20,"710,000.00",2022-02-10,2022-03-04,0
53,PTTGC,7.28,1.75,0.00,2.00,0.00,3.75,"12,000","45,000.00",51.50,"618,000.00",2022-02-28,2022-04-22,0
37,STA,7.10,0.65,0.00,1.25,0.00,1.90,"22,500","42,750.00",26.75,"601,875.00",2022-04-19,2022-05-06,1
26,TMT,11.11,0.70,0.00,0.60,0.00,1.30,"27,000","35,100.00",11.70,"315,900.00",2022-04-18,2022-05-06,1
72,TISCO,7.19,7.15,0.00,0.00,0.00,7.15,"4,000","28,600.00",99.50,"398,000.00",2022-04-29,2022-05-19,1
71,HREIT,8.04,0.17,0.17,0.17,0.17,0.70,"40,100","27,877.52",8.65,"346,865.00",2022-03-04,2022-03-30,0
13,SCCC,11.65,9.00,0.00,9.00,0.00,18.00,"1,200","21,600.00",154.50,"185,400.00",2022-02-24,2022-04-08,0


In [13]:
div_price[y_ovr_X & in_port].div_amt.sum()

543651.52

In [14]:
dividend = div_price[y_ovr_X & in_port].div_amt.sum()
cost = div_price[y_ovr_X & in_port].amount.sum()
yld = round(dividend / cost * 100,2)
(dividend, cost, yld)

(543651.52, 6274040.0, 8.67)

In [15]:
file_name = 'DIV-OVR-X%.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

div_price[y_ovr_X & in_port].sort_values(['yield','name'],ascending=[False,True]).to_csv(data_file, index=False)
div_price[y_ovr_X & in_port].sort_values(['yield','name'],ascending=[False,True]).to_csv(output_file, index=False)
div_price[y_ovr_X & in_port].sort_values(['yield','name'],ascending=[False,True]).to_csv(box_file, index=False)
div_price[y_ovr_X & in_port].sort_values(['yield','name'],ascending=[False,True]).to_csv(one_file, index=False)

In [16]:
div_price[~y_ovr_X & in_port].sort_values(['yield','name'],ascending=[False,True]).style.format(format_dict)

,name,yield,Q4,Q3,Q2,Q1,DIVIDEND,shares,div_amt,current,amount,XDATE,PAIDDATE,ACTUAL
16,AIMIRT,6.89,0.22,0.22,0.22,0.22,0.88,"10,000","8,822.00",12.80,"128,000.00",2022-03-14,2022-03-28,0
0,WHART,6.77,0.12,0.26,0.19,0.19,0.76,"20,000","15,156.00",11.20,"224,000.00",2022-03-09,2022-03-25,0
15,NER,6.10,0.36,0.00,0.07,0.00,0.43,"27,000","11,610.00",7.05,"190,350.00",2022-04-19,2022-05-06,1
44,SPALI,5.66,0.75,0.00,0.50,0.00,1.25,"9,000","11,250.00",22.10,"198,900.00",2022-04-27,2022-05-12,1
57,BCH,5.61,1.00,0.00,0.20,0.00,1.20,"15,000","18,000.00",21.40,"321,000.00",2022-05-10,2022-05-25,1
62,TPIPL,5.49,0.06,0.00,0.03,0.00,0.09,"60,000","5,400.00",1.64,"98,400.00",2022-03-09,2022-05-17,0
17,PTT,5.06,0.80,0.00,1.20,0.00,2.00,"9,000","18,000.00",39.50,"355,500.00",2022-03-03,2022-04-29,0
9,SCC,4.84,10.00,0.00,8.50,0.00,18.50,900,"16,650.00",382.00,"343,800.00",2022-04-07,2022-04-26,1
49,SIS,3.10,1.20,0.00,0.00,0.00,1.20,"4,500","5,400.00",38.75,"174,375.00",2022-03-10,2022-05-13,0
47,SYNEX,2.69,0.52,0.00,0.18,0.00,0.70,"18,000","12,600.00",26.00,"468,000.00",2022-03-10,2022-05-05,0


In [17]:
div_price[~y_ovr_X & in_port].sort_values(['div_amt','name'],ascending=[False,True]).style.format(format_dict)

,name,yield,Q4,Q3,Q2,Q1,DIVIDEND,shares,div_amt,current,amount,XDATE,PAIDDATE,ACTUAL
46,KCE,2.45,1.00,0.00,0.60,0.00,1.60,"13,000","20,800.00",65.25,"848,250.00",2022-03-21,2022-05-09,0
57,BCH,5.61,1.00,0.00,0.20,0.00,1.20,"15,000","18,000.00",21.40,"321,000.00",2022-05-10,2022-05-25,1
17,PTT,5.06,0.80,0.00,1.20,0.00,2.00,"9,000","18,000.00",39.50,"355,500.00",2022-03-03,2022-04-29,0
9,SCC,4.84,10.00,0.00,8.50,0.00,18.50,900,"16,650.00",382.00,"343,800.00",2022-04-07,2022-04-26,1
0,WHART,6.77,0.12,0.26,0.19,0.19,0.76,"20,000","15,156.00",11.20,"224,000.00",2022-03-09,2022-03-25,0
47,SYNEX,2.69,0.52,0.00,0.18,0.00,0.70,"18,000","12,600.00",26.00,"468,000.00",2022-03-10,2022-05-05,0
15,NER,6.10,0.36,0.00,0.07,0.00,0.43,"27,000","11,610.00",7.05,"190,350.00",2022-04-19,2022-05-06,1
44,SPALI,5.66,0.75,0.00,0.50,0.00,1.25,"9,000","11,250.00",22.10,"198,900.00",2022-04-27,2022-05-12,1
16,AIMIRT,6.89,0.22,0.22,0.22,0.22,0.88,"10,000","8,822.00",12.80,"128,000.00",2022-03-14,2022-03-28,0
49,SIS,3.10,1.20,0.00,0.00,0.00,1.20,"4,500","5,400.00",38.75,"174,375.00",2022-03-10,2022-05-13,0


In [18]:
dividend = div_price[~y_ovr_X & in_port].div_amt.sum()
cost = div_price[~y_ovr_X & in_port].amount.sum()
yld = round(dividend / cost * 100,2)
(dividend, cost, yld)

(146883.0, 3802300.0, 3.86)